### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [313]:
# Dependencies and Setup
import pandas as pd
import csv


# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools--

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [314]:
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [315]:

schoolcount = school_data_complete["school_name"].nunique()
studentcount = school_data_complete["student_name"].count()
totalbudget = school_data_complete["budget"].sum()
avgmath = school_data_complete["maths_score"].mean()
avgread = school_data_complete["reading_score"].mean()
permaths = len(school_data_complete[school_data_complete["maths_score"]>=50])/studentcount*100
perread = len(school_data_complete[school_data_complete["reading_score"]>=50])/studentcount*100
percomb = len(school_data_complete[(school_data_complete["maths_score"]>=50) & (school_data_complete["reading_score"]>=50)])/studentcount*100

sum_df=pd.DataFrame([{"schoolcount":schoolcount}, {"studentcount":studentcount}, {"totalbudget":totalbudget},{"avgmath":avgmath},{"avgread":avgread},{"% Passing Maths":permaths},{"% Passing Reading":perread}, {"% Overall Passing":percomb}])
sum_df=sum_df.sum().to_frame().T
sum_df


,schoolcount,studentcount,totalbudget,avgmath,avgread,% Passing Maths,% Passing Reading,% Overall Passing
0,15.0,39170.0,8.293233e+10,70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [316]:
#Create a data frame for mean scores+totalstudents+budget
grouped_school_dfmean = school_data_complete.groupby(["school_name"]).mean()
grouped_school_dfmean = grouped_school_dfmean.drop(['Student ID','year','School ID'], axis=1)

#Create a dataframe for schooltypes
grouped_school_dftype = school_data_complete.groupby(["school_name"]).max()
grouped_school_dftype = grouped_school_dftype.drop(['size', 'Student ID', 'year', 'student_name', 'gender', 'Student ID', 'reading_score', 'maths_score','budget'], axis=1)

#Create a dataframe for counting math score passes
grouped_school_mathpasses = school_data_complete[school_data_complete["maths_score"]>49].groupby(["school_name"]).count()

#Create a dataframe for counting read score passes
grouped_school_readpasses = school_data_complete[school_data_complete["reading_score"]>49].groupby(["school_name"]).count()

#Create a dataframe for counting both passes
grouped_school_ovrpass = school_data_complete[school_data_complete["reading_score"]>49]
grouped_school_ovrpass = grouped_school_ovrpass[grouped_school_ovrpass["maths_score"]>49].groupby(["school_name"]).count()

#Merge dataframes and create % Passing columns
grouped_school_dfmerge = pd.merge(grouped_school_dfmean, grouped_school_dftype, how="outer", on=["school_name"])
grouped_school_dfmerge['% Passing Maths'] = grouped_school_mathpasses["maths_score"]/grouped_school_dfmerge['size']
grouped_school_dfmerge['% Passing Reading'] = grouped_school_readpasses["maths_score"]/grouped_school_dfmerge['size']
grouped_school_dfmerge['% Overall Passing'] = grouped_school_ovrpass["maths_score"]/grouped_school_dfmerge['size']

grouped_school_dfmerge.head()

,reading_score,maths_score,size,budget,School ID,type,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,71.008842,72.352894,4976.0,3124928.0,7,Government,0.916399,0.873794,0.800844
Cabrera High School,71.359526,71.657158,1858.0,1081356.0,6,Independent,0.908504,0.890743,0.807858
Figueroa High School,69.077993,68.698542,2949.0,1884411.0,1,Government,0.816548,0.828077,0.676501
Ford High School,69.572472,69.091274,2739.0,1763916.0,13,Government,0.824388,0.822198,0.674699
Griffin High School,71.245232,71.788147,1468.0,917500.0,4,Independent,0.912125,0.884877,0.813351


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [317]:
#Sort dataframe by % Passing in Descending order
top_schools=grouped_school_dfmerge.sort_values(by=["% Overall Passing"], ascending=False)
top_schools.head(5)

,reading_score,maths_score,size,budget,School ID,type,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,71.245232,71.788147,1468.0,917500.0,4,Independent,0.912125,0.884877,0.813351
Cabrera High School,71.359526,71.657158,1858.0,1081356.0,6,Independent,0.908504,0.890743,0.807858
Bailey High School,71.008842,72.352894,4976.0,3124928.0,7,Government,0.916399,0.873794,0.800844
Wright High School,70.969444,72.047222,1800.0,1049400.0,10,Independent,0.917778,0.866667,0.797222
Rodriguez High School,70.935984,72.047762,3999.0,2547363.0,11,Government,0.907977,0.873968,0.794199


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [318]:
#Sort dataframe by % Passing in Ascending order
bottom_schools=grouped_school_dfmerge.sort_values(by=["% Overall Passing"], ascending=True)
bottom_schools.head(5)

,reading_score,maths_score,size,budget,School ID,type,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,69.186408,68.874865,4635.0,3022020.0,3,Government,0.809493,0.818770,0.663646
Huang High School,68.910525,68.935207,2917.0,1910635.0,0,Government,0.816935,0.814535,0.667124
Johnson High School,69.039277,68.843100,4761.0,3094650.0,12,Government,0.820626,0.819786,0.671918
Wilson High School,68.876916,69.170828,2283.0,1319574.0,5,Independent,0.827858,0.812965,0.674551
Ford High School,69.572472,69.091274,2739.0,1763916.0,13,Government,0.824388,0.822198,0.674699


## Maths Scores by Year

In [319]:
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [320]:
#Create a dataframe on schoolname/year and drop non necessary columns
maths_score_by_year = school_data_complete.groupby(["school_name","year"]).mean().drop(["size","budget","School ID", "Student ID", "reading_score"], axis=1).rename(columns={"maths_score":"average maths score"})

maths_score_by_year.head()


average maths score
school_name         year                     
Bailey High School  9               72.493827
                    10              71.897498
                    11              72.374900
                    12              72.675097
Cabrera High School 9               72.321970

## Reading Score by Year

* Perform the same operations as above for reading scores

In [321]:
#Create a dataframe on schoolname/year and drop non necessary columns
read_score_by_year = school_data_complete.groupby(["school_name","year"]).mean().drop(["size","budget","School ID", "Student ID", "maths_score"], axis=1).rename(columns={"reading_score":"average reading score"})

read_score_by_year.head()



average reading score
school_name         year                       
Bailey High School  9                 70.901920
                    10                70.848265
                    11                70.317346
                    12                72.195525
Cabrera High School 9                 71.172348

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [322]:
grouped_school_dfmerge.head()

,reading_score,maths_score,size,budget,School ID,type,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,71.008842,72.352894,4976.0,3124928.0,7,Government,0.916399,0.873794,0.800844
Cabrera High School,71.359526,71.657158,1858.0,1081356.0,6,Independent,0.908504,0.890743,0.807858
Figueroa High School,69.077993,68.698542,2949.0,1884411.0,1,Government,0.816548,0.828077,0.676501
Ford High School,69.572472,69.091274,2739.0,1763916.0,13,Government,0.824388,0.822198,0.674699
Griffin High School,71.245232,71.788147,1468.0,917500.0,4,Independent,0.912125,0.884877,0.813351


In [323]:
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]


school_spending_df=grouped_school_dfmerge
school_spending_df["Spending Ranges (Per Student)"] = grouped_school_dfmerge["budget"]/grouped_school_dfmerge["size"]
school_spending_df = school_spending_df.rename(columns={"maths_score":"Average Maths Score","reading_score":"Average Reading Score"}).drop(["size", "School ID"], axis=1)

spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Maths Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Maths"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]



## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type